In [10]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

In [41]:

SANDBOX_DIR = "/Users/aidand/dev/auto-llama/sandbox"


def create_file(path, content):
    # Create any directories that don't exist
    os.makedirs(os.path.dirname(os.path.join(SANDBOX_DIR, path)), exist_ok=True)
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Created file {os.path.join(SANDBOX_DIR, path)}")

# Does the same thing as create_file - but nice to have a separate function for updating files
# So the LLM has the option to update files if it wants to - if that makes more sense than creating a new file
def update_file(path, content):
    os.makedirs(os.path.dirname(os.path.join(SANDBOX_DIR, path)), exist_ok=True)
    with open(os.path.join(SANDBOX_DIR, path), "w") as f:
        f.write(content)
    print(f"Updated file {os.path.join(SANDBOX_DIR, path)}")

def delete_file(path):
    # If the file doesn't exist, don't do anything
    if not os.path.exists(os.path.join(SANDBOX_DIR, path)):
        print(f"Tried to delete file {os.path.join(SANDBOX_DIR, path)} but it does not exist")
        return
    os.remove(os.path.join(SANDBOX_DIR, path))
    print(f"Deleted file {os.path.join(SANDBOX_DIR, path)}")

# def make_dir(path):
#     os.makedirs(os.path.join(SANDBOX_DIR, path), exist_ok=True)
#     print(f"Created directory {os.path.join(SANDBOX_DIR, path)}")

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "create_file",
            "description": "Create a file with the given name and content. If there are any directories that don't exist, create them.",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the file to create",
                    },
                    "content": {
                        "type": "string",
                        "description": "The content of the file to create",
                    },
                },
                "required": ["path", "content"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "update_file",
            "description": "Update a file with the given name and content. If the file does not exist, create it.",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to update",
                },
                "content": {
                    "type": "string",
                    "description": "The content of the file to update",
                },
            },
            "required": ["path", "content"],
        },
    },
    {
        "type": "function",
        "function": {
            "name": "delete_file",
            "description": "Delete a file with the given path. If the file does not exist, do nothing.",
        },
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": "The relative path to the file to delete",
                },
            },
            "required": ["path"],
        },
    },
    # {
    #     "type": "function",
    #     "function": {
    #         "name": "make_dir",
    #         "description": "Create a directory with the given name",
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "path": {
    #                     "type": "string",
    #                     "description": "The relative path to the directory to create",
    #                 },
    #             },
    #             "required": ["path"],
    #         },
    #     },
    # },
]

def run_tool(tool_call):
    arguments = json.loads(tool_call.function.arguments)
    if tool_call.function.name == "create_file":
        create_file(arguments["path"], arguments["content"])
    elif tool_call.function.name == "update_file":
        update_file(arguments["path"], arguments["content"])
    elif tool_call.function.name == "delete_file":
        delete_file(arguments["path"])
    # elif tool_call.function.name == "make_dir":
    #     make_dir(arguments["path"])


In [48]:
from openai import OpenAI
from pydantic import BaseModel
import json

PROGRAM_OBJECTIVE="download a PDF from the internet and convert it to a text file."

CODER_AGENT_SYSTEM_PROMPT=f"""
You are a software engineer who is writing code to build a small python codebase that will {PROGRAM_OBJECTIVE}.
"""

from typing import List

class Plan(BaseModel):
    steps: List[str]

client = OpenAI(api_key=OPENAI_API_KEY)

if os.path.exists(SANDBOX_DIR):
    # Clear the contents of the directory
    for item in os.listdir(SANDBOX_DIR):
        item_path = os.path.join(SANDBOX_DIR, item)
        if os.path.isfile(item_path):
            os.unlink(item_path)
        elif os.path.isdir(item_path):
            import shutil
            shutil.rmtree(item_path)
else:
    os.makedirs(SANDBOX_DIR)

# Coding agent creates a step by step plan
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": CODER_AGENT_SYSTEM_PROMPT},
        {"role": "user", "content": f"""
         Create a step by step plan to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.
         You have 4 different operations you can perform. create_file(path, content), update_file(path, content), delete_file(path).
         Limit your step by step plan to only these operations.
         Please include a README.md file in the root of the codebase that describes the codebase and how to run it.
         Please include a requirements.txt file in the root of the codebase that describes the dependencies of the codebase.
         """},
    ],
    tools=TOOLS,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Plan",
            "description": f"A plan to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.",
            "strict": True,
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["steps"],
                "additionalProperties": False,
            },
        },
    },
)
plan = json.loads(response.choices[0].message.content)
print("Coder Agent - Plan")
for step in plan["steps"]:
    print(step)
print("-" * 100)

# Being more concrete
# print("Coder Agent - Being more concrete")
# for step in plan["steps"]:
#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=[
#             {"role": "system", "content": CODER_AGENT_SYSTEM_PROMPT},
#             {"role": "user", "content": """
#              You have come up with the following plan: {json.dumps(plan)}.
#              Please be more concrete about the steps you will take to complete the task of creating a codebase that will {PROGRAM_OBJECTIVE}.
#              For example:

#              Step: Create a README.md file in the 'my_project' directory with content describing the project and how to run it
#              """},
#         ],
#     )
#     print(response.choices[0].message.content)

# Coding agent executes the plan
print("Coder Agent - Executing Plan")
for step in plan["steps"]:
    prompt = f"""
        You have 4 different operations you can perform. create_file(path, content), update_file(path, content), delete_file(path), make_dir(path).
        Please perform the following operation: {step}
        """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": CODER_AGENT_SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
        tools=TOOLS,
    )
    message = response.choices[0].message
    if message.content:
        print("Not enough information to run tool: ", message.content)
    else:
        tool_call = message.tool_calls[0]
        run_tool(tool_call)

# Gets a concatenated string of all the files in the codebase and their contents
def get_codebase_contents():
    contents = ""
    for root, dirs, files in os.walk(SANDBOX_DIR):
        for file in files:
            # concatenate the file name
            contents += f"file: {file}:\n"
            with open(os.path.join(root, file), "r") as f:
                contents += f.read()
            contents += "EOF\n\n"
    return contents

# Review agent reviews the code
# REVIEWER_AGENT_SYSTEM_PROMPT=f"""
# You are a software engineer who is reviewing the codebase that was created by another software engineer.
# The program is meant to {PROGRAM_OBJECTIVE}.
# Please review the codebase and make sure it is correct.
# Please provide a list of changes you would like to make to the codebase.
# """

Coder Agent - Plan
Create a directory named 'pdf_to_text_converter' for the codebase.
Inside the 'pdf_to_text_converter' directory, create a Python script named 'converter.py' that will contain the logic to download a PDF and convert it to a text file.
Create a README.md file in the root of the 'pdf_to_text_converter' directory to describe the codebase and provide instructions on how to run it.
Create a requirements.txt file in the root of the 'pdf_to_text_converter' directory to list the dependencies needed (e.g., requests, PyPDF2).
In 'converter.py', import required libraries, set up a function to download the PDF from a given URL, save it locally, and implement the logic to convert the PDF into text.
Test the functionality of 'converter.py' to ensure it works as expected, handling any issues like file not found or URL errors gracefully.
----------------------------------------------------------------------------------------------------
Coder Agent - Executing Plan
Created directory 